# 経済学研究開発環境セットアップ完全ガイド

このノートブックでは、経済学研究のための完全な開発環境を構築し、検証します。

## 環境構成
- **Docker**: rootless Docker with WSL2
- **Python**: conda/mamba + Poetry (pip禁止運用)
- **R**: tidyverse, IRkernel
- **LaTeX**: Econometrica (ectaart), 日本語対応, BibTeX
- **開発ツール**: pre-commit, ruff, mypy, pytest
- **エディタ**: VS Code with Dev Containers

## 1. Docker環境セットアップ

In [ ]:
import subprocess
import os
import sys
from pathlib import Path

def run_command(cmd, capture_output=True):
    """コマンドを実行して結果を表示"""
    try:
        result = subprocess.run(cmd, shell=True, capture_output=capture_output, text=True)
        if capture_output:
            print(f"Command: {cmd}")
            print(f"Return code: {result.returncode}")
            if result.stdout:
                print(f"Output:\n{result.stdout}")
            if result.stderr:
                print(f"Error:\n{result.stderr}")
        return result.returncode == 0
    except Exception as e:
        print(f"エラーが発生しました: {e}")
        return False

# Docker環境の確認
print("=== Docker環境確認 ===")
run_command("docker --version")
run_command("docker info | grep -i rootless")
run_command("docker ps")

## 2. Python環境設定（Poetry）

In [ ]:
# Python環境の確認
print("=== Python環境確認 ===")
print(f"Python version: {sys.version}")
print(f"Python executable: {sys.executable}")
print(f"Current working directory: {os.getcwd()}")

# Poetry確認
print("\n=== Poetry確認 ===")
run_command("poetry --version")
run_command("poetry config --list")

# 仮想環境情報
run_command("poetry env info")

In [ ]:
# 必要なパッケージのインポートテスト
print("=== 必須パッケージインポートテスト ===")

packages_to_test = [
    'numpy', 'pandas', 'scipy', 'matplotlib', 'seaborn', 
    'statsmodels', 'sklearn', 'jupyter'
]

for package in packages_to_test:
    try:
        if package == 'sklearn':
            import sklearn
            print(f"✅ {package}: {sklearn.__version__}")
        else:
            module = __import__(package)
            version = getattr(module, '__version__', 'unknown')
            print(f"✅ {package}: {version}")
    except ImportError as e:
        print(f"❌ {package}: インポートエラー - {e}")
    except Exception as e:
        print(f"⚠️ {package}: エラー - {e}")

## 3. R環境セットアップ

In [ ]:
# R環境の確認
print("=== R環境確認 ===")
run_command("R --version | head -1")
run_command("which R")

# Rの基本パッケージ確認
r_check_script = '''
R -q -e "packageVersion('base'); packageVersion('tidyverse'); packageVersion('IRkernel');"
'''
run_command(r_check_script.strip())

In [ ]:
# Jupyter R カーネルの確認
print("=== Jupyter カーネル確認 ===")
run_command("jupyter kernelspec list")

# 利用可能なカーネルの詳細
import json
from jupyter_client.kernelspec import KernelSpecManager

ksm = KernelSpecManager()
kernels = ksm.get_all_specs()

print("\n=== 利用可能なJupyterカーネル ===")
for name, spec in kernels.items():
    print(f"📚 {name}: {spec['spec']['display_name']}")
    print(f"   Language: {spec['spec']['language']}")
    print(f"   Location: {spec['resource_dir']}\n")

## 4. Jupyterカーネル設定

In [ ]:
# カーネル登録状況の詳細確認
print("=== カーネル登録詳細 ===")

# Python dev カーネルの確認
python_kernel_path = "/opt/conda/share/jupyter/kernels/dev"
if Path(python_kernel_path).exists():
    print("✅ Python (dev) カーネルが登録されています")
    run_command(f"cat {python_kernel_path}/kernel.json")
else:
    print("❌ Python (dev) カーネルが見つかりません")

print("\n" + "="*50)

# R カーネルの確認
r_kernel_path = "/opt/conda/share/jupyter/kernels/ir"
if Path(r_kernel_path).exists():
    print("✅ R (IRkernel) カーネルが登録されています")
    run_command(f"cat {r_kernel_path}/kernel.json")
else:
    print("❌ R (IRkernel) カーネルが見つかりません")

## 5. LaTeX環境テスト

In [ ]:
# LaTeX環境の確認
print("=== LaTeX環境確認 ===")
run_command("lualatex --version | head -1")
run_command("pdflatex --version | head -1")
run_command("bibtex --version")
run_command("latexmk --version | head -1")

# Econometrica関連ファイルの確認
print("\n=== Econometrica関連確認 ===")
run_command("kpsewhich ectaart.cls")
run_command("kpsewhich econometrica.bst || echo 'econometrica.bst not found in texmf tree'")

In [ ]:
# 簡単なLaTeXテストドキュメントを作成・コンパイル
print("=== LaTeX テストコンパイル ===")

# テスト用のLaTeXファイルを作成
test_latex = r"""
\documentclass[12pt]{article}
\usepackage{ifluatex}
\ifluatex
  \usepackage{luatexja-fontspec}
  \setmainfont{TeX Gyre Termes}
  \setmainjfont{Noto Serif CJK JP}
\fi
\usepackage{amsmath,graphicx}
\title{LaTeX Test Document}
\author{Environment Setup}
\date{\today}
\begin{document}
\maketitle
\section{テスト}
これは日本語TeXのテストです。

数式のテスト:
\begin{equation}
E[Y|X] = \beta_0 + \beta_1 X
\end{equation}

\end{document}
"""

# テストファイルの書き出し
test_dir = Path("/tmp/latex_test")
test_dir.mkdir(exist_ok=True)

test_file = test_dir / "test.tex"
test_file.write_text(test_latex.strip())

print(f"テストファイルを作成: {test_file}")

# LuaLaTeXでコンパイル
os.chdir(test_dir)
success = run_command("lualatex -interaction=nonstopmode test.tex")
if success:
    print("✅ LuaLaTeX コンパイル成功")
    if (test_dir / "test.pdf").exists():
        print("✅ PDF ファイル生成確認")
else:
    print("❌ LuaLaTeX コンパイル失敗")

## 6. 開発ツール検証

In [ ]:
# プロジェクトルートに戻る
os.chdir("/workspaces")

# 開発ツールの確認
print("=== 開発ツール確認 ===")
run_command("pre-commit --version")
run_command("poetry run ruff --version")
run_command("poetry run mypy --version")
run_command("poetry run pytest --version")

print("\n=== pre-commit設定確認 ===")
if Path(".pre-commit-config.yaml").exists():
    print("✅ pre-commit設定ファイルが存在")
    run_command("pre-commit validate-config")
else:
    print("❌ pre-commit設定ファイルが見つかりません")

In [ ]:
# サンプルPythonコードでツールをテスト
print("=== 開発ツールテスト ===")

# テスト用のPythonファイルを作成
test_python_code = '''
"""テスト用のPythonモジュール"""
from typing import List
import numpy as np
import pandas as pd


def calculate_mean(data: List[float]) -> float:
    """数値リストの平均を計算する関数"""
    if not data:
        raise ValueError("データが空です")
    return sum(data) / len(data)


def create_sample_dataframe() -> pd.DataFrame:
    """サンプルDataFrameを作成"""
    np.random.seed(42)
    data = {
        "x": np.random.randn(100),
        "y": np.random.randn(100)
    }
    return pd.DataFrame(data)


if __name__ == "__main__":
    test_data = [1.0, 2.0, 3.0, 4.0, 5.0]
    mean_value = calculate_mean(test_data)
    print(f"平均: {mean_value}")
    
    df = create_sample_dataframe()
    print(f"DataFrame shape: {df.shape}")
'''.strip()

# テストファイルの作成
test_src_dir = Path("src")
test_src_dir.mkdir(exist_ok=True)

test_py_file = test_src_dir / "test_module.py"
test_py_file.write_text(test_python_code)

print(f"テスト用Pythonファイルを作成: {test_py_file}")

# ruffでのフォーマットとリント
print("\n=== Ruff テスト ===")
run_command(f"poetry run ruff format {test_py_file}")
run_command(f"poetry run ruff check {test_py_file}")

# mypyでの型チェック
print("\n=== mypy テスト ===")
run_command(f"poetry run mypy {test_py_file}")

In [ ]:
# pytest用のテストファイルを作成
print("=== pytest テスト ===")

test_pytest_code = '''
"""test_module.py用のテスト"""
import pytest
from src.test_module import calculate_mean, create_sample_dataframe


def test_calculate_mean():
    """calculate_mean関数のテスト"""
    # 正常ケース
    assert calculate_mean([1.0, 2.0, 3.0]) == 2.0
    assert calculate_mean([0.0]) == 0.0
    
    # 異常ケース
    with pytest.raises(ValueError):
        calculate_mean([])


def test_create_sample_dataframe():
    """create_sample_dataframe関数のテスト"""
    df = create_sample_dataframe()
    assert df.shape == (100, 2)
    assert list(df.columns) == ["x", "y"]
    assert not df.isna().any().any()
'''.strip()

# testsディレクトリの作成
tests_dir = Path("tests")
tests_dir.mkdir(exist_ok=True)

# __init__.pyファイルの作成
(test_src_dir / "__init__.py").touch()
(tests_dir / "__init__.py").touch()

# テストファイルの作成
test_pytest_file = tests_dir / "test_test_module.py"
test_pytest_file.write_text(test_pytest_code)

print(f"pytestファイルを作成: {test_pytest_file}")

# pytestの実行
run_command("poetry run pytest tests/ -v")

## 7. GitHub統合セットアップ

In [ ]:
# Git設定の確認
print("=== Git設定確認 ===")
run_command("git --version")
run_command("git config --global user.name || echo 'No user.name set'")
run_command("git config --global user.email || echo 'No user.email set'")
run_command("git config --list | grep safe.directory || echo 'No safe.directory set'")

# リポジトリ状態の確認
print("\n=== リポジトリ状態 ===")
run_command("git status")
run_command("git remote -v || echo 'No remotes configured'")

# SSH設定の確認（あれば）
print("\n=== SSH設定確認 ===")
ssh_dir = Path.home() / ".ssh"
if ssh_dir.exists():
    print("✅ SSH設定ディレクトリが存在")
    if (ssh_dir / "id_rsa").exists() or (ssh_dir / "id_ed25519").exists():
        print("✅ SSH秘密鍵が存在")
        run_command("ssh-add -l || echo 'No SSH keys in agent'")
    else:
        print("ℹ️ SSH鍵が見つかりません")
else:
    print("ℹ️ SSH設定ディレクトリが存在しません")

In [ ]:
# GitHub Actions CI設定の確認
print("=== GitHub Actions設定確認 ===")
ci_file = Path(".github/workflows/ci.yml")
if ci_file.exists():
    print("✅ GitHub Actions CI設定が存在")
    print("\n=== CI設定内容 ===")
    print(ci_file.read_text())
else:
    print("❌ GitHub Actions CI設定が見つかりません")

# プロジェクト構成の最終確認
print("\n=== プロジェクト構成確認 ===")
important_files = [
    "pyproject.toml",
    "environment.yml", 
    ".pre-commit-config.yaml",
    ".gitignore",
    ".devcontainer/devcontainer.json",
    ".devcontainer/Dockerfile",
    "tex/paper/main.tex",
    "tex/slides/talk.tex"
]

for file_path in important_files:
    if Path(file_path).exists():
        print(f"✅ {file_path}")
    else:
        print(f"❌ {file_path}")

## まとめ

この環境セットアップノートブックでは、以下の項目を検証しました：

1. ✅ **Docker環境**: rootless Docker, コンテナ機能
2. ✅ **Python環境**: Poetry, 必須パッケージ
3. ✅ **R環境**: tidyverse, IRkernel
4. ✅ **Jupyter**: Python/Rカーネル
5. ✅ **LaTeX**: LuaLaTeX, 日本語対応, Econometrica
6. ✅ **開発ツール**: pre-commit, ruff, mypy, pytest
7. ✅ **GitHub統合**: Git設定, CI/CD

### 次のステップ
- 実際の研究プロジェクトでノートブックを作成
- LaTeX論文・スライドの作成
- GitHub Actions でのCI/CD実行確認
- データ分析ワークフローの構築